# 初始化

In [ ]:
import os
import time
import threading
from http.server import HTTPServer, SimpleHTTPRequestHandler
from local_packages import *
from dotenv import load_dotenv
from queue import Queue
import concurrent.futures
import random
import json
import openai
from concurrent.futures import ThreadPoolExecutor, as_completed

# 加载环境变量
dotenv_path = os.path.join(os.getcwd(), '.env')

# 设置项目根目录和图片目录
project_root = os.path.dirname(dotenv_path)

service_type = 'huida'

def initialize_service(service_type):
    if service_type in ['zhipu', None]:
        version = 'glm-3-turbo'
        #'glm-4' 'glm-4v' 'glm-3-turbo'
        service = GLMService(version)
    elif service_type in ['qwen']:
        version = 'long'
        service=QwenService(version)
    elif service_type in ['kimi']:
        version = '8k'
        #'8k'1M/12￥ '32k'1M/24￥ '128k'1M/60￥
        service = KimiService(version)
    elif service_type in ['huida']:
        version = 'gpt-4o'
        #'8k'1M/12￥ '32k'1M/24￥ '128k'1M/60￥
        service = HuidaService(version)
    elif service_type in ['sensetime']:
        version = 'SenseChat'
        #SenseChat SenseChat-32K SenseChat-128K SenseChat-Turbo SenseChat-FunctionCall
        service = SenseService(version=version)
    else:
        raise ValueError('未知的服务类型')
    
    return service

service = initialize_service(service_type)

js=JSProcessor()

class ParseError(Exception):
    def __init__(self, code, message="解析失败"):
        self.code = code
        self.message = message
        super().__init__(self.message)


In [ ]:
import os

# 定义根文件夹和模型路径
root_folder = 'Task20_Product-Tech-Fix'
model_path = r'D:\Joining\Models\Text2Vec_base_zh'

product_data_json_path = os.path.join(root_folder, "product_demand.json")

tech_data_json_path = os.path.join(root_folder, "tech_function.json")

step_1_process_folder = os.path.join(root_folder, "step_1_process")

step_1_processed_json_path = os.path.join(step_1_process_folder, "step_1_processed.json")

step_1_temp_json_path = os.path.join(step_1_process_folder, "step_1_temp.json")

step_2_process_folder = os.path.join(root_folder, "step_2_process")

step_2_temp_json_path = os.path.join(step_2_process_folder, "step_2_temp.json")

step_2_processed_json_path = os.path.join(step_2_process_folder, "step_2_processed.json")

step_3_process_folder = os.path.join(root_folder, "step_3_process")

step_3_processed_json_path = os.path.join(step_3_process_folder, "step_3_processed.json")

step_3_temp_json_path = os.path.join(step_3_process_folder, "step_3_temp.json")

step_4_process_folder = os.path.join(root_folder, "step_4_process")

step_4_processed_json_path = os.path.join(step_4_process_folder, "step_4_processed.json")

step_4_temp_json_path = os.path.join(step_4_process_folder, "step_4_temp.json")

step_5_process_folder = os.path.join(root_folder, "step_5_process")

step_5_processed_json_path = os.path.join(step_5_process_folder, "step_5_processed.json")

step_5_temp_json_path = os.path.join(step_5_process_folder, "step_5_temp.json")

step_6_process_folder = os.path.join(root_folder, "step_6_process")

step_6_processed_json_path = os.path.join(step_6_process_folder, "step_6_processed.json")

step_6_temp_json_path = os.path.join(step_6_process_folder, "step_6_temp.json")
# 创建所有需要的目录
directories = [
    root_folder,
    step_1_process_folder,
    step_2_process_folder,
    step_3_process_folder,
    step_4_process_folder,
    step_5_process_folder,
    step_6_process_folder
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)

# 创建所有需要的文件
files = [
    product_data_json_path,
    tech_data_json_path,
    step_1_processed_json_path,
    step_2_processed_json_path,
    step_3_processed_json_path,
    step_4_processed_json_path,
    step_5_processed_json_path,
    step_6_processed_json_path,
    step_1_temp_json_path,
    step_2_temp_json_path,
    step_3_temp_json_path,
    step_4_temp_json_path,
    step_5_temp_json_path,
    step_6_temp_json_path
]

for file in files:
    if not os.path.exists(file):
        with open(file, 'w', encoding='utf-8') as f:
            if file.endswith('.json'):
                f.write('{}')  # 写入空的JSON对象
            elif file.endswith('.txt'):
                f.write('')  # 创建空的txt文件

print("所有目录和文件已成功创建。")


In [ ]:
task_func()